# 01 - Inspect and Clean Data

This notebook is used to explore and clean the raw datasets for FindMyBorough.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def load_and_preview(file_path, sheet=None, header=0):
    """
    Load and preview a dataset (.csv or .xls/.xlsx).
    
    Parameters:
    - file_path (str): Path to the dataset file.
    - sheet (int or str, optional): Sheet index or name for Excel files.
    - header (int): Row index to use as column names for Excel/CSV.

    Returns:
    - pd.DataFrame
    """
    print(f"Loading file: {file_path}")
    
    if file_path.endswith(".csv"):
        df = pd.read_csv(file_path, header=header)
        
    elif file_path.endswith(".xls") or file_path.endswith(".xlsx"):
        xls = pd.ExcelFile(file_path)
        print("Available sheets:", xls.sheet_names)
        
        if sheet is None:
            sheet = 0  # Default to first sheet
        
        df = xls.parse(sheet, header=header)
        
    else:
        raise ValueError("Unsupported file type. Use .csv or .xls/.xlsx.")
    
    print("\nPreview:")
    display(df.head())
    print("\nInfo:")
    display(df.info())
    print("\nColumns:", df.columns.tolist())
    
    return df


In [3]:
def standardize_borough_names(series):
    """
    Normalize borough names:
    - Lowercase
    - Strip whitespace
    - Replace '&' with 'and'
    - Replace known variants (only if needed)
    """
    replacements = {
        'westminster city': 'westminster'
    }

    return (
        series
        .astype(str)
        .str.strip()
        .str.lower()
        .str.replace('&', 'and')
        .replace(replacements)
    )

In [4]:
valid_boroughs = [
    'barking and dagenham', 'barnet', 'bexley', 'brent', 'bromley', 'camden', 'city of london',
    'croydon', 'ealing', 'enfield', 'greenwich', 'hackney', 'hammersmith and fulham',
    'haringey', 'harrow', 'havering', 'hillingdon', 'hounslow', 'islington',
    'kensington and chelsea', 'kingston upon thames', 'lambeth', 'lewisham', 'merton',
    'newham', 'redbridge', 'richmond upon thames', 'southwark', 'sutton',
    'tower hamlets', 'waltham forest', 'wandsworth', 'westminster'
]

def filter_valid_boroughs(df, borough_column='borough', valid_boroughs=None):
    """
    Filter a DataFrame to include only valid (normalized) London boroughs.
    
    Parameters:
    - df: pd.DataFrame — the dataset to filter
    - borough_column: str — name of the column containing borough names
    - valid_boroughs: list[str] — list of valid boroughs in lowercase

    Returns:
    - pd.DataFrame — filtered DataFrame
    """
    if valid_boroughs is None:
        raise ValueError("valid_boroughs list is required")

    df[borough_column] = df[borough_column].str.strip().str.lower()
    
    return df[df[borough_column].isin(valid_boroughs)].copy()


## Rental Prices

In [5]:
df_rent = load_and_preview("../data/raw/voa-average-rent-borough.xls", sheet="Raw data", header=2)

Loading file: ../data/raw/voa-average-rent-borough.xls
Available sheets: ['Metadata', 'Summary', 'Pivot Table', 'Raw data']

Preview:


,Year,Quarter,Code,Area,Category,Count of rents,Average,Lower quartile,Median,Upper quartile
0,2011,Q2,E09000001,City of London,Room,-,-,-,-,-
1,2011,Q2,E09000002,Barking and Dagenham,Room,92,336,282,347,390
2,2011,Q2,E09000003,Barnet,Room,945,450,399,433,500
3,2011,Q2,E09000004,Bexley,Room,119,390,347,390,433
4,2011,Q2,E09000005,Brent,Room,344,469,390,457,550



Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6160 entries, 0 to 6159
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Year            6160 non-null   int64 
 1   Quarter         6160 non-null   object
 2   Code            6160 non-null   object
 3   Area            6160 non-null   object
 4   Category        6160 non-null   object
 5   Count of rents  6160 non-null   object
 6   Average         6160 non-null   object
 7   Lower quartile  6160 non-null   object
 8   Median          6160 non-null   object
 9   Upper quartile  6160 non-null   object
dtypes: int64(1), object(9)
memory usage: 481.4+ KB


None


Columns: ['Year', 'Quarter', 'Code', 'Area', 'Category', 'Count of rents', 'Average', 'Lower quartile', 'Median', 'Upper quartile']


In [6]:
# Filter rows where the property type is 'All categories'
df_all = df_rent[df_rent['Category'] == 'All categories']

# Group by borough and calculate the median rent
df_rent_clean = df_all.groupby('Area')['Median'].median().reset_index()

# Rename columns to match the feature naming convention
df_rent_clean.columns = ['borough', 'avg_rent']

# Normalize borough names and filter only London ones
df_rent_clean['borough'] = standardize_borough_names(df_rent_clean['borough'])
df_rent_clean = filter_valid_boroughs(df_rent_clean, 'borough', valid_boroughs)
df_rent_clean.head()

,borough,avg_rent
0,barking and dagenham,900.5
1,barnet,1264.0
2,bexley,850.0
3,brent,1314.0
4,bromley,1025.0


## Crime

In [7]:
df_crime = load_and_preview("../data/raw/M1045_MonthlyCrimeDashboard_TNOCrimeData.xlsx")

Loading file: ../data/raw/M1045_MonthlyCrimeDashboard_TNOCrimeData.xlsx
Available sheets: ['MPS_MonthlyCrimeDashboard_TNOCr']

Preview:


,Month_Year,Area Type,Borough_SNT,Area name,Area code,Offence Group,Offence Subgroup,Measure,Financial Year,FY_FYIndex,Count,Refresh Date
0,2021-03-01,Borough,Barking and Dagenham,Barking and Dagenham,E09000002,DRUG OFFENCES,TRAFFICKING OF DRUGS,Positive Outcomes,fy20-21,20-21_01,6,2025-03-05
1,2021-03-01,Borough,Barking and Dagenham,Barking and Dagenham,E09000002,ARSON AND CRIMINAL DAMAGE,ARSON,Offences,fy20-21,20-21_01,5,2025-03-05
2,2021-03-01,Borough,Barking and Dagenham,Barking and Dagenham,E09000002,BURGLARY,BURGLARY - BUSINESS AND COMMUNITY,Positive Outcomes,fy20-21,20-21_01,2,2025-03-05
3,2021-03-01,Borough,Barking and Dagenham,Barking and Dagenham,E09000002,POSSESSION OF WEAPONS,POSSESSION OF WEAPONS,Positive Outcomes,fy20-21,20-21_01,11,2025-03-05
4,2021-03-01,Borough,Barking and Dagenham,Barking and Dagenham,E09000002,VEHICLE OFFENCES,INTERFERING WITH A MOTOR VEHICLE,Positive Outcomes,fy20-21,20-21_01,1,2025-03-05



Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 789526 entries, 0 to 789525
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Month_Year        789526 non-null  datetime64[ns]
 1   Area Type         789526 non-null  object        
 2   Borough_SNT       789526 non-null  object        
 3   Area name         789020 non-null  object        
 4   Area code         786377 non-null  object        
 5   Offence Group     789526 non-null  object        
 6   Offence Subgroup  789526 non-null  object        
 7   Measure           789526 non-null  object        
 8   Financial Year    789526 non-null  object        
 9   FY_FYIndex        789526 non-null  object        
 10  Count             789526 non-null  int64         
 11  Refresh Date      789526 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(9)
memory usage: 72.3+ MB


None


Columns: ['Month_Year', 'Area Type', 'Borough_SNT', 'Area name', 'Area code', 'Offence Group', 'Offence Subgroup', 'Measure', 'Financial Year', 'FY_FYIndex', 'Count', 'Refresh Date']


In [8]:
# Keep only rows where the measure is 'Offences'
df_crime_filtered = df_crime[df_crime['Measure'] == 'Offences']

# Group by borough and sum total crimes
df_crime_clean = df_crime_filtered.groupby('Area name')['Count'].sum().reset_index()

# Rename columns
df_crime_clean.columns = ['borough', 'total_crimes']
print(df_crime_clean.sort_values("total_crimes", ascending=False).head(1))  # cantidad de filas
print(df_crime_clean[df_crime_clean["borough"].str.lower().str.contains("kensington")])

# Normalize borough names and filter only London ones
df_crime_clean['borough'] = standardize_borough_names(df_crime_clean['borough'])
df_crime_clean = filter_valid_boroughs(df_crime_clean, 'borough', valid_boroughs)

         borough  total_crimes
700  Westminster        314934
                    borough  total_crimes
351  Kensington and Chelsea         85957
693         West Kensington          3290


## Age group

In [9]:
df_age = load_and_preview("../data/raw/londonboroughs.xlsx", sheet="Age London Boroughs")

Loading file: ../data/raw/londonboroughs.xlsx
Available sheets: ['Metadata', 'Household Type Boroughs', 'Ethnic Group Boroughs', 'London HH Type', 'Age London Boroughs', 'Age London', 'London Ethnic Group']

Preview:


,Lower tier local authorities Code,Lower tier local authorities,Age (18 categories) Code,Age (18 categories),Observation
0,E09000001,City of London,1,Aged 4 years and under,213
1,E09000001,City of London,2,Aged 5 to 9 years,159
2,E09000001,City of London,3,Aged 10 to 15 years,206
3,E09000001,City of London,4,Aged 16 to 19 years,184
4,E09000001,City of London,5,Aged 20 to 24 years,965



Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 594 entries, 0 to 593
Data columns (total 5 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   Lower tier local authorities Code  594 non-null    object
 1   Lower tier local authorities       594 non-null    object
 2   Age (18 categories) Code           594 non-null    int64 
 3   Age (18 categories)                594 non-null    object
 4   Observation                        594 non-null    int64 
dtypes: int64(2), object(3)
memory usage: 23.3+ KB


None


Columns: ['Lower tier local authorities Code', 'Lower tier local authorities', 'Age (18 categories) Code', 'Age (18 categories)', 'Observation']


In [10]:
df_age.columns = df_age.columns.str.strip()

df_age.rename(columns={
    "Lower tier local authorities": "borough",
    "Age (18 categories)": "age_group",
    "Observation": "population"
}, inplace=True)

df_age["borough"] = df_age["borough"].str.strip().str.lower()
df_age["age_group"] = df_age["age_group"].str.strip()

young_ages = [
    "Aged 20 to 24 years",
    "Aged 25 to 29 years",
    "Aged 30 to 34 years"
]

# Total population per borough
df_total_pop = df_age.groupby("borough")["population"].sum().reset_index()
df_total_pop.rename(columns={"population": "total_population"}, inplace=True)

# Young population per borough
df_young = df_age[df_age["age_group"].isin(young_ages)]
df_young_pop = df_young.groupby("borough")["population"].sum().reset_index()
df_young_pop.rename(columns={"population": "young_population"}, inplace=True)

# Merge and calculate ratio
df_age_features = pd.merge(df_total_pop, df_young_pop, on="borough", how="inner")
df_age_features["young_ratio"] = df_age_features["young_population"] / df_age_features["total_population"]

# Keep only final columns
df_age_features = df_age_features[["borough", "young_ratio"]]

In [11]:
# Normalize borough names and filter only valid London boroughs
df_age_features['borough'] = standardize_borough_names(df_age_features['borough'])
df_age_clean = filter_valid_boroughs(df_age_features, 'borough', valid_boroughs)

# Preview
df_age_clean.head()

,borough,young_ratio
0,barking and dagenham,0.212506
1,barnet,0.207044
2,bexley,0.184581
3,brent,0.246455
4,bromley,0.167963
